In [4]:
import pandas as pd

In [34]:
def fix_name(name):
    if name in name_fixes:
        return name_fixes[name]
    else:
        return name

name_fixes = {
    "Corey Taylor": "Corey Blackett-Taylor",
    "Craig Carl Curran": "Craig Curran",
    "Chris Edwards": "Christian Edwards",
    "Dave Nugent": "David Nugent",
    "Dylan Mottley Henry": "Dylan Mottley-Henry",
    "Ian Thomas-Moore": "Ian Moore",
    "John-Louis Akpa Akpro": "Jean-Louis Akpa Akpro",
    "John Morrissey": "Johnny Morrissey",
    "Jonathon Margetts": "Johnny Margetts",
    "Joseph Maguire": "Joe Maguire",
    "Kaylden Brown": "Kayleden Brown",
    "Lewis Sinnot": "Lewis Sinnott",
    "Lateef Elford Alliyu": "Lateef Elford-Alliyu",
    "Michael Jackson": "Mike Jackson",
    "Richard Sutton": "Ritchie Sutton",
    "Robert Taylor": "Rob Taylor",
    "Samuel Taylor": "Sam Taylor",
    "Steven O'Leary": "Stephen O'Leary"
}

In [35]:
sb_match_apps = pd.read_csv(
    "https://raw.githubusercontent.com/petebrown/update-player-stats/main/data/players_df.csv",
    parse_dates = ["game_date"]
).rename(columns = {
    "sb_game_id": "game_id",
    "sb_player_id": "player_id",
})

sb_match_apps.game_id = sb_match_apps.game_id.str.replace("tpg", "").astype(int)

In [61]:
sb_game_ids = sb_match_apps[["game_id", "game_date", "season"]].drop_duplicates().sort_values(by = ["game_date"]).reset_index(drop = True)

sb_game_ids.head(3)

,game_id,game_date,season
0,240309,1996-08-17,1996/97
1,243200,1996-08-21,1996/97
2,242383,1996-08-23,1996/97


In [62]:
sb_player_ids = sb_match_apps[["player_id", "player_name"]].drop_duplicates().reset_index(drop = True)
sb_player_ids.player_name = sb_player_ids.player_name.apply(fix_name)

sb_player_ids.head(3)

,player_id,player_name
0,128729,Jake Burton
1,18605,Peter Clarke
2,103555,Josh Dacres-Cogley


In [63]:
results_df = pd.read_csv("https://raw.githubusercontent.com/petebrown/update-results/main/data/results_df.csv", parse_dates = ["game_date"])

results_df.head(1)

,season,game_date,opposition,venue,score,home_team,away_team,outcome,home_goals,away_goals,...,attendance,stadium,league_tier,generic_comp,game_type,goal_diff,ssn_game_no,ssn_comp_game_no,weekday,manager
0,2022/23,2023-05-08,Northampton Town,H,0-1,Tranmere Rovers,Northampton Town,L,0.0,1.0,...,8225.0,Prenton Park,4.0,Football League,League,-1.0,53,46,Monday,Ian Dawes


In [69]:
game_nos = results_df[["game_date", "season", "ssn_game_no"]].rename(columns = {"ssn_game_no": "game_no"}).sort_values(["game_date"]).reset_index(drop = True)

game_nos.head(3)

,game_date,season,game_no
0,1921-08-27,1921/22,1
1,1921-09-03,1921/22,2
2,1921-09-10,1921/22,3


In [64]:
managers_df = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/managers.csv", parse_dates=["date_from", "date_to"])

managers_df.head(3)

,manager_name,date_from,date_to,role
0,Bert Cooke,1921-08-27,1935-04-30,Secretary/Manager
1,Jack Carr,1935-07-02,1936-10-22,Manager
2,No manager,1936-10-23,1936-11-30,Manager


In [65]:
fa_trophy_goals = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/fa-trophy/fa_trophy_goals.csv", parse_dates = ["game_date"])

fa_trophy_goals.head(3)

,game_date,player_name,minute,penalty,own_goal
0,2015-12-12,James Norwood,45,0,0
1,2015-12-12,James Norwood,82,0,0
2,2016-12-10,Ritchie Sutton,26,0,0


In [73]:
sb_goals = pd.read_csv("https://raw.githubusercontent.com/petebrown/scrape-goals/main/data/goals.csv")
sb_goals = sb_goals[sb_goals["goal_type"] == "for"]
sb_goals.player_name = sb_goals.player_name.apply(fix_name)
sb_goals = sb_goals.merge(sb_game_ids, on="game_id", how="left")[["game_date", "player_name", "minute", "penalty", "own_goal"]].sort_values(["game_date", "minute"])
sb_goals

,game_date,player_name,minute,penalty,own_goal
1713,1996-08-17,Johnny Morrissey,22,0,0
1714,1996-08-21,John Aldridge,30,0,0
1715,1996-08-21,Ivano Bonetti,56,0,0
1716,1996-08-23,John Aldridge,24,0,0
1718,1996-08-23,Kenny Irons,51,0,0
...,...,...,...,...,...
1828,2023-04-18,Richard O'Donnell,19,0,1
1829,2023-04-18,Josh Hawkes,56,1,0
1830,2023-04-22,Josh Hawkes,33,1,0
1831,2023-04-22,Rhys Hughes,48,0,0


In [77]:
all_goals = pd.read_csv("https://raw.githubusercontent.com/petebrown/complete-record/main/output/scorers-long.csv").merge(game_nos, how = "left", on = ["season", "game_no"])[["season", "player_name", "goals_scored"]]



,season,player_name,goals_scored
0,1921/22,Tom Stuart,1
1,1921/22,Charles Milnes,1
2,1921/22,Fred Groves,1
3,1921/22,John Ford,1
4,1921/22,John Prentice,1
...,...,...,...
5946,2022/23,Harvey Saunders,1
5947,2022/23,Josh Hawkes,1
5948,2022/23,Josh Hawkes,1
5949,2022/23,Rhys Hughes,1
